In [1]:
#import all libraries

import tensorflow as tf
from tensorflow.keras import layers,models


In [2]:

dataset = tf.keras.preprocessing.image_dataset_from_directory(

    directory = "PlantVillage",
    batch_size=32,
    seed = 123,
    shuffle = True,
    image_size = [255,255]

)

Found 900 files belonging to 3 classes.


In [3]:
classname = dataset.class_names

In [4]:
len(dataset)

29

In [5]:
import numpy as np
for batch_size,label_size in dataset.take(1):
    print("image size:", batch_size.shape)
    print("batch:", label_size.numpy)
    print("image",batch_size[0])

image size: (32, 255, 255, 3)
batch: <bound method _EagerTensorBase.numpy of <tf.Tensor: shape=(32,), dtype=int32, numpy=
array([1, 2, 2, 2, 0, 1, 2, 1, 0, 1, 0, 0, 1, 1, 2, 2, 0, 0, 1, 1, 2, 2,
       1, 0, 2, 0, 0, 0, 1, 2, 0, 2], dtype=int32)>>
image tf.Tensor(
[[[146.89827  134.89827  138.89827 ]
  [117.32479  105.32479  109.32479 ]
  [159.80559  147.80559  151.80559 ]
  ...
  [189.00586  179.00586  178.00586 ]
  [188.99608  178.99608  177.99608 ]
  [188.99413  178.99413  177.99413 ]]

 [[125.09177  113.09177  117.09177 ]
  [153.73062  141.73062  145.73062 ]
  [135.74919  123.74919  127.74919 ]
  ...
  [187.0177   177.0177   176.0177  ]
  [187.01765  177.01765  176.01765 ]
  [186.02547  176.02547  175.02547 ]]

 [[131.32344  119.323456 123.323456]
  [126.39447  114.39447  118.39447 ]
  [143.41678  131.41678  135.41678 ]
  ...
  [190.0195   180.0195   179.0195  ]
  [190.00006  180.00006  179.00006 ]
  [190.       180.       179.      ]]

 ...

 [[133.22902  120.22902  127.22902 ]
  

In [6]:
train_size = 0.8
len(dataset) * train_size

23.200000000000003

In [7]:
train_ds = dataset.take(54)

In [8]:
test_ds = dataset.skip(54)

In [9]:
len(test_ds)

0

In [10]:
val_size = 0.1
len(dataset)*val_size


2.9000000000000004

In [11]:
val_ds = dataset.take(6)
test_ds = test_ds.skip(6)


In [12]:
len(train_ds)

29

In [13]:
len(test_ds)

0

In [14]:
len(val_ds)

6

In [15]:
def dataset_partion_tf(ds,train_split = 0.8, val_split = 0.1,test_split=0.1,shuffle=True,shuffle_size=10000):
    assert(train_split + test_split+val_split) == 1
    ds_size = len(ds)

    if shuffle:
        ds = ds.shuffle(shuffle_size,seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)

    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)

    return train_ds, val_ds,test_ds
train_ds,val_ds,test_ds = dataset_partion_tf(dataset)

    


In [16]:
len(train_ds)

23

In [17]:
len(test_ds)


4

In [18]:
len(val_ds)

2

In [19]:
train_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
val_ds=val_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
test_ds=test_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)


In [20]:
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

resize_and_rescale = Sequential([
    layers.Resizing(255,255),
    layers.Rescaling(1.0 / 255)
])

In [21]:
# data augumentation
data_augmentation = Sequential([

    layers.RandomFlip("horizontal_and_vertical"),  # Randomly flip images horizontally and vertically
    layers.RandomRotation(0.2),  # Randomly rotate images by 20%

])

In [22]:
train_ds=train_ds.map(lambda x,y:(data_augmentation(x,training=True),y)).prefetch(buffer_size = tf.data.AUTOTUNE)

In [23]:
model = models.Sequential([
    resize_and_rescale,
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(255, 255, 3)), 
    layers.MaxPooling2D((2, 2)),  

    # Convolutional layer 2
    layers.Conv2D(64, (3, 3), activation='relu'),  
    layers.MaxPooling2D((2, 2)),

    # Convolutional layer 3
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    

    # Flatten layer
    layers.Flatten(),  # Converts the 3D feature maps to 1D feature vectors

    # Fully connected (Dense) layer
    layers.Dense(128, activation='relu'),  # 128 neurons

    # Output layer
    layers.Dense(64,activation='softmax')  
])




d:\Project\myenv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [24]:
model.compile(optimizer='adam',
loss='sparse_categorical_crossentropy',
metrics=['accuracy'])


In [25]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [26]:
history = model.fit(
    train_ds,
    epochs=20,
    batch_size=32,
    validation_data=val_ds
)

Epoch 1/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.2773 - loss: 2.4844 - val_accuracy: 0.6562 - val_loss: 0.8287
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 23s 983ms/step - accuracy: 0.7025 - loss: 0.7549 - val_accuracy: 0.8750 - val_loss: 0.3218
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 22s 937ms/step - accuracy: 0.8910 - loss: 0.2962 - val_accuracy: 0.7969 - val_loss: 0.4215
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 22s 939ms/step - accuracy: 0.8951 - loss: 0.2570 - val_accuracy: 0.8750 - val_loss: 0.1905
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 22s 922ms/step - accuracy: 0.9159 - loss: 0.2175 - val_accuracy: 0.9688 - val_loss: 0.1434
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 22s 928ms/step - accuracy: 0.9244 - loss: 0.2060 - val_accuracy: 1.0000 - val_loss: 0.0717
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.9534 - loss: 0.1354 - val_accuracy: 0.9531 - val_loss: 0.0989
Epoch 8/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 24s 1s/step - accuracy: 0.9463 - loss: 0.1470 - val_accuracy: 0.9

In [27]:
model.save("model.h5")

In [28]:
model = tf.keras.models.load_model('model.h5')

In [29]:
for img,label in test_ds.take(1):
    first_img = img[0].numpy().astype('uint8')
    first_label = label[0].numpy()
    pred_img = model.predict(img)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 399ms/step


In [30]:
def predict(model,img):
    img_array = tf.keras.preprocessing.image.img_to_array(image[i].numpy())
    img_array = tf.expand_dims(img_array,0)

    predictions = model.predict(img_array)
    predicted_class = classname[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])),2)
    return predicted_class,confidence